In [37]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from ai_models.shared import load_processed_data, load_train_with_validation_data
from constants import numeric_features, categorical_features

In [38]:

df = load_processed_data()
df = df.dropna(subset=["Target encoded"])
X = df.drop(columns=["Target", "Target encoded"])
y = df["Target encoded"]

In [39]:
num_features = [c for c in numeric_features if c in X.columns]
cat_features = [c for c in categorical_features if c in X.columns]

In [40]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

In [41]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [42]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])


In [43]:
X_train, X_val, X_test, y_train, y_val, y_test = load_train_with_validation_data(X, y)

In [45]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC()
}

for name, model in models.items():
    pipeline = Pipeline([
        ("preprocessor", full_pipeline),
        ("classifier", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_val_pred = pipeline.predict(X_val)
    print(f"\n{name}")
    print("Test Set Metrics:")
    print(classification_report(y_test, y_pred))
    print("Validation Set Metrics:")
    print(classification_report(y_val, y_val_pred))


Logistic Regression
Test Set Metrics:
              precision    recall  f1-score   support

        -1.0       0.92      0.86      0.89       217
         1.0       0.91      0.95      0.93       328

    accuracy                           0.92       545
   macro avg       0.92      0.91      0.91       545
weighted avg       0.92      0.92      0.92       545

Validation Set Metrics:
              precision    recall  f1-score   support

        -1.0       0.93      0.83      0.87       214
         1.0       0.90      0.96      0.93       330

    accuracy                           0.91       544
   macro avg       0.91      0.89      0.90       544
weighted avg       0.91      0.91      0.91       544


Decision Tree
Test Set Metrics:
              precision    recall  f1-score   support

        -1.0       0.83      0.82      0.83       217
         1.0       0.88      0.89      0.89       328

    accuracy                           0.86       545
   macro avg       0.86      0.8